In [1]:
!pip install selenium --upgrade

PRODUCT SCRAPING

In [2]:
import selenium
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By

In [3]:
import os
print(os.getcwd())

/Users/sukanyasemwal


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(executable_path='/Users/sukanyasemwal/Downloads/chromedriver_mac_arm64')

In [7]:
browser = webdriver.Chrome(executable_path='/Users/sukanyasemwal/Downloads/chromedriver_mac_arm64')
browser.get('https://www.amazon.in/')

In [8]:
product_name = input("Enter the product name: ")

Enter the product name:  guitar


In [9]:
search_box = browser.find_element(By.ID, 'twotabsearchtextbox')
search_box.send_keys(product_name)

In [10]:
search_button = browser.find_element(By.XPATH, "//input[@value='Go']")
search_button.click()

In [11]:
time.sleep(10) # Wait for page to load
search_results = browser.find_elements(By.CSS_SELECTOR, '.s-result-item')
for result in search_results:
    print(result.text)

Shop for Yamaha Acoustic Guitars
Shop Yamaha Musical Instruments 
Yamaha FS100C Acoustic Guitar,…
 22
Yamaha F280 Acoustic Rosew…
 2,315
Yamaha F310-TBS Right Handed A…
 64
Sponsored
Showing products near you, with fast delivery
See all products, across price ranges.
RESULTS
Sponsored
Kadence Frontier Jumbo Semi Acoustic Guitar With Die Cast Keys, Super Combo (Bag, 1 pack Strings, Strap, Picks, Capo (Black)
4.0
(1,323)
Limited time deal
₹6,899
₹9,750 (29% off)
Get it by Tomorrow, 25 February
FREE Delivery by Amazon
Sponsored
Kadence Frontier guitar with Online Guitar learning course , Wine Red Acoustic Guitar with Die Cast Keys, Set of Strings, Strap, Picks and Bag (Wine Red, Acoustic)
4.0
(1,897)
Limited time deal
₹4,899
₹7,680 (36% off)
Get it by Tomorrow, 25 February
FREE Delivery by Amazon
Sponsored
Vault DA20 Dreadnought Acoustic Guitar With Gig-Bag, Picks, Strings Set, String Winder, Strap, Polishing Cloth And Allen Key
4.0
(13)
₹4,669
₹10,000 (53% off)
Get it by Tomorrow, 25 Feb

DATAFRAME AND CSV FILE

In [1]:
import selenium
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
import os
print(os.getcwd())

/Users/sukanyasemwal


In [3]:
from selenium import webdriver
# Activating the chrome browser
driver=webdriver.Chrome('/Users/sukanyasemwal/Downloads/chromedriver_mac_arm64')

In [4]:
driver.get("https://www.amazon.in/")

In [5]:
search_term = input("Enter product to search on Amazon.in: ")

Enter product to search on Amazon.in: guitar


In [6]:
search_box = driver.find_element(By.ID, "twotabsearchtextbox")
search_box.send_keys(search_term)
search_box.send_keys(Keys.RETURN)

In [7]:
#scrape all links for products
links=[]
for x in range(0,3):
    link=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in link:
        links.append(i.get_attribute("href"))
        
    click_on_next_page=driver.find_element(By.XPATH,"//a[text()='Next']")
    click_on_next_page.click()
    time.sleep(3)
print(len(links))

182


In [8]:
#scrape require data from webpages
names=[]
brands=[]
prices=[]
returns=[]
expected_delivery=[]
availability=[]

for i in links:
    try:
        driver.get(i)
        try:
            name=driver.find_element(By.XPATH,'//h1[@id="title"]/span')
            names.append(name.text)
        except NoSuchElementException:
            names.append('-')

        try:
            brand=driver.find_element(By.XPATH,'//div[@class="a-section a-spacing-none"]/a')
            brands.append(brand.text)
        except NoSuchElementException:
            brands.append('-')
            
        try:
            price = driver.find_element(By.XPATH, '//span[@id="priceblock_ourprice"]')
            prices.append(price.text)
        except NoSuchElementException:
            prices.append('-')

        try:
            return_policy = driver.find_element(By.XPATH, '//div[@id="RETURNS_POLICY"]/span[2]')
            returns.append(return_policy.text)
        except NoSuchElementException:
            returns.append('-')

        try:
            delivery = driver.find_element(By.XPATH, '//div[@id="ddmDeliveryMessage"]/b')
            expected_delivery.append(delivery.text)
        except NoSuchElementException:
            expected_delivery.append('-')

        try:
            avail = driver.find_element(By.XPATH, '//*[@id="p_n_availability-title"]/span')
            availability.append(avail.text)
        except NoSuchElementException:
            availability.append('-')
            
    except:
        print('Error occured while scraping data for:',i)

# print the scraped data
print(names)
print(brands)
print(prices)
print(returns)
print(expected_delivery)
print(availability)

['Kadence Frontier guitar with Online Guitar learning course , Wine Red Acoustic Guitar with Die Cast Keys, Set of Strings, Strap, Picks and Bag (Wine Red, Acoustic)', 'Vault Traveller 34 Inch Acoustic Guitar With Truss Rod, Gig-Bag, Strap, Strings, Polishing Cloth, String Winder, Picks & E-Book - Natural Gloss', "Hontral S4 String Decor Guitar Children's Musical Instrument Educational Toy Small Guitar for Beginners Kids Child - 23Inch", "RK Shop String Acoustic Guitar , Learning Toys for Girl & Boys,Musical Instrument Educational Guitar Toy for Beginners Kids 17'' - ( Pack of 1 )", 'Juârez Acoustic Guitar, 38 Inch Cutaway, 038C with Bag, Strings, Pick and Strap, Black', 'Yamaha F280 Acoustic Rosewood Guitar (Natural, Beige)', 'Intern INT-38C Natural Linden Wood Acoustic Guitar Kit with Carry Bag & Picks', 'Juarez Lindenwood Acoustic Guitar, 38 Inches Curved Body Cutaway, 38CUR With Bag, Strings, Pick And Strap (Black)', 'JUAREZ JRZ38C 38 Inches Lindenwood Right Handed Acoustic Guitar 

In [9]:
all_data = {
    'Name': names,
    'Brand': brands,
    'Price': prices,
    'Return Policy': returns,
    'Expected Delivery': expected_delivery,
    'Availability': availability
    'Product_url': product_url}

In [10]:
df = pd.DataFrame(all_data, columns=['Name', 'Brand', 'Price', 'Return Policy', 'Expected Delivery', 'Availability', ])
print(df)

                                                  Name  \
0    Kadence Frontier guitar with Online Guitar lea...   
1    Vault Traveller 34 Inch Acoustic Guitar With T...   
2    Hontral S4 String Decor Guitar Children's Musi...   
3    RK Shop String Acoustic Guitar , Learning Toys...   
4    Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...   
..                                                 ...   
177  JUAREZ Arpeggio 39 Inch Acoustic Guitar Kit, S...   
178  Roscoe 4 String Acoustic Guitar Ukulele, Music...   
179  Kadence AstroMan Electric Guitar, 21 FRETS, RO...   
180  Medellin 38" Acoustic Guitar Purple Burst Carb...   
181  JUAREZ Arpéggio 104.14 cm (41 Inch) Acoustic G...   

                       Brand Price Return Policy Expected Delivery  \
0    Visit the Kadence Store     -             -    Monday, Feb 27   
1      Visit the VAULT Store     -             -                 -   
2             Brand: Hontral     -             -                 -   
3             Brand: RK

In [11]:
df.to_csv('guitars.csv', index=False)

In [12]:
driver.quit()

IMAGES-FRUITS

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import os
import time

In [14]:
# Set up the driver
driver = webdriver.Chrome()
driver.get("https://images.google.com/")

In [15]:
# Define the search terms
search_terms = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]

# Loop over each search term and scrape 10 images
for term in search_terms:
    # Find the search bar and enter the search term
    search_bar = driver.find_element(By.NAME, "q")
    search_bar.clear()
    search_bar.send_keys(term)
    search_bar.send_keys(Keys.RETURN)

    # Scroll down to load more images
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Find the image elements and get their source URLs
    image_elements = driver.find_elements(By.CSS_SELECTOR, "img.rg_i")
    image_urls = [image_element.get_attribute("src") for image_element in image_elements]

    # Create a folder to store the images and urls
    folder_name = term
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Download the images and save them to the folder
    with open(f"{folder_name}/{term}_urls.txt", "w") as url_file:
        for i, image_url in enumerate(image_urls[:10]):
            try:
                file_name = f"{folder_name}/{term}{i}.jpg"
                urllib.request.urlretrieve(image_url, file_name)
                print(f"Image {i+1} for {term} downloaded")
                url_file.write(f"{image_url}\n")
            except:
                print(f"Image {i+1} for {term} failed to download")

Image 1 for fruits downloaded
Image 2 for fruits downloaded
Image 3 for fruits downloaded
Image 4 for fruits downloaded
Image 5 for fruits downloaded
Image 6 for fruits downloaded
Image 7 for fruits downloaded
Image 8 for fruits downloaded
Image 9 for fruits downloaded
Image 10 for fruits downloaded
Image 1 for cars downloaded
Image 2 for cars downloaded
Image 3 for cars downloaded
Image 4 for cars downloaded
Image 5 for cars downloaded
Image 6 for cars downloaded
Image 7 for cars downloaded
Image 8 for cars downloaded
Image 9 for cars downloaded
Image 10 for cars downloaded
Image 1 for Machine Learning downloaded
Image 2 for Machine Learning downloaded
Image 3 for Machine Learning downloaded
Image 4 for Machine Learning downloaded
Image 5 for Machine Learning downloaded
Image 6 for Machine Learning downloaded
Image 7 for Machine Learning downloaded
Image 8 for Machine Learning downloaded
Image 9 for Machine Learning downloaded
Image 10 for Machine Learning downloaded
Image 1 for Guita

In [17]:
driver.quit()

SMARTPHONE

In [22]:
import selenium
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd

In [23]:
import os
print(os.getcwd())

/Users/sukanyasemwal


In [24]:
from selenium import webdriver
# Activating the chrome browser
driver=webdriver.Chrome('/Users/sukanyasemwal/Downloads/chromedriver_mac_arm64')

In [25]:
# Navigate to Flipkart
driver.get("https://www.flipkart.com/")

In [26]:
search_term = input("Enter product to search on flipkart.com: ")

Enter product to search on flipkart.com: smartphone


In [27]:
search_box = driver.find_element(By.XPATH, "/html/body/div/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input")
search_box.send_keys(search_term)
search_box.send_keys(Keys.RETURN)

In [28]:
# scrape all links for products on the first page of smartphone search on Flipkart
links=[]
link_elements = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for link in link_elements:
    links.append(link.get_attribute("href"))

print(len(links))

24


In [29]:
# scrape the required details for each smartphone on the page
data = []
for product in driver.find_elements(By.XPATH, '//div[@class="_2kHMtA"]'):
    details = {
        'Brand Name': '-',
        'Smartphone Name': '-',
        'Colour': '-',
        'RAM': '-',
        'Storage(ROM)': '-',
        'Primary Camera': '-',
        'Secondary Camera': '-',
        'Display Size': '-',
        'Battery Capacity': '-',
        'Price': '-',
        'Product URL': '-'
    }
    try:
        # extract the brand name and smartphone name
        name = product.find_element(By.XPATH, './/div[@class="_4rR01T"]')
        name_parts = name.text.split(' ')
        details['Brand Name'] = name_parts[0]
        details['Smartphone Name'] = ' '.join(name_parts[1:])
        
        # extract the remaining details
        specs = product.find_elements(By.XPATH, './/ul[@class="_1xgFaf"]/li')
        for spec in specs:
            label = spec.find_element(By.XPATH, './/span[1]').text
            value = spec.find_element(By.XPATH, './/span[2]').text
            if label == 'Color':
                details['Colour'] = value
            elif label == 'RAM':
                details['RAM'] = value
            elif label == 'Internal Storage':
                details['Storage(ROM)'] = value
            elif label == 'Primary Camera':
                details['Primary Camera'] = value
            elif label == 'Secondary Camera':
                details['Secondary Camera'] = value
            elif label == 'Display Size':
                details['Display Size'] = value
            elif label == 'Battery Capacity':
                details['Battery Capacity'] = value
                
        # extract the price and product URL
        price = product.find_element(By.XPATH, './/div[@class="_30jeq3 _1_WHN1"]')
        details['Price'] = price.text[1:].replace(',', '')
        url = name.find_element(By.XPATH, './/a').get_attribute('href')
        details['Product URL'] = url
    except:
        pass
    
    data.append(details)

In [30]:
# save the results in a DataFrame 
df = pd.DataFrame(data)
print(df)

   Brand Name                      Smartphone Name Colour RAM Storage(ROM)  \
0        vivo          T1 44W (Starry Sky, 128 GB)      -   -            -   
1      realme            C30 (Bamboo Green, 32 GB)      -   -            -   
2        vivo     T1 44W (Midnight Galaxy, 128 GB)      -   -            -   
3      realme               C30 (Lake Blue, 32 GB)      -   -            -   
4      realme            C30 (Bamboo Green, 32 GB)      -   -            -   
5      realme               C30 (Lake Blue, 32 GB)      -   -            -   
6      realme             C30 (Denim Black, 32 GB)      -   -            -   
7        vivo     T1 44W (Midnight Galaxy, 128 GB)      -   -            -   
8        POCO             C31 (Shadow Gray, 32 GB)      -   -            -   
9       REDMI          10 (Caribbean Green, 64 GB)      -   -            -   
10      APPLE            iPhone 11 (White, 128 GB)      -   -            -   
11       POCO         M4 Pro 5G (Cool Blue, 64 GB)      -   -   

In [31]:
# save the result to csv file
df.to_csv('smartphone_data.csv', index=False)

In [32]:
driver.quit()

GEOSPATIAL COORDINATES

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [34]:
# launch the web browser and open Google Maps
driver = webdriver.Chrome('/Users/sukanyasemwal/Downloads/chromedriver_mac_arm64')
driver.get("https://www.google.com/maps/")

In [35]:
# wait for the page to load
time.sleep(5)

# find the search box and enter the city name
search_box = driver.find_element(By.ID, "searchboxinput")
search_box.send_keys('New York')

In [36]:
# click on the search button
search_button = driver.find_element(By.ID, "searchbox-searchbutton")
search_button.click()

# wait for the search results to load
time.sleep(5)

In [37]:
# extract the latitude and longitude coordinates from the URL
url = driver.current_url
coords_start = url.index('@') + 1
coords_end = url.index('/', coords_start)
coords = url[coords_start:coords_end].split(',')
latitude = coords[0]
longitude = coords[1]

print("Latitude:", latitude)
print("Longitude:", longitude)

Latitude: 12.9129842
Longitude: 77.6421466


In [38]:
# close the web browser
driver.quit()

BEST GAMING LAPTOPS

In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [40]:
# Create a webdriver instance and navigate to the digit.in laptop section
driver = webdriver.Chrome()
driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html')

In [41]:
# Get the list of laptop links
laptop_links = []
link_elements = driver.find_elements(By.XPATH, '//div[@class="right-container"]//a')
for link in link_elements:
    laptop_links.append(link.get_attribute("href"))

# Create a list to store the laptop information
laptop_info = []

# Loop through each laptop link and extract the required information
for link in laptop_links:
    driver.get(link)
    time.sleep(5)  # Wait for the page to load
    try:
        laptop_name = driver.find_element(By.XPATH, '//div[@class="heading-wraper"]/h1')
    except:
        laptop_name = None
    specs = driver.find_elements(By.XPATH, '//div[@class="specifications"]/table/tbody/tr')
    laptop_specs = {}
    for spec in specs:
        try:
            key = spec.find_element(By.XPATH, './/td[1]').text.strip()
            value = spec.find_element(By.XPATH, './/td[3]').text.strip()
            laptop_specs[key] = value
        except:
            pass
    laptop_info.append({'Name': laptop_name.text if laptop_name else None, 'Specifications': laptop_specs})

# Convert the list of laptop information to a pandas dataframe and save to a CSV file
df = pd.DataFrame(laptop_info)
print(df)
df.to_csv('gaming_laptops.csv', index=False)

                                                 Name Specifications
0   MSI Titan GT77 12UHS 12th Gen Core i9-12900HX ...             {}
1   MSI Titan GT77 12UHS 12th Gen Core i9-12900HX ...             {}
2   MSI Titan GT77 12UHS 12th Gen Core i9-12900HX ...             {}
3                                                None             {}
4                 New Msi Laptops Price List in India             {}
..                                                ...            ...
65                                               None             {}
66                                               None             {}
67                                               None             {}
68                                               None             {}
69                                               None             {}

[70 rows x 2 columns]


In [42]:
# quit the driver
driver.quit()

FORBES-WORLD BILLIONARES

In [43]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [44]:
# Create a new Chrome browser instance
driver = webdriver.Chrome()

In [45]:
# Navigate to Forbes website and maximize the window
driver.get("https://www.forbes.com/")
driver.maximize_window()

In [47]:
# Open lists for billionaires
lists = driver.find_element(By.XPATH, '/html/body/div[1]/header/nav/div[1]/div/div/div[1]')
lists.click()

# Open billionaires dropdown
billionaires = driver.find_element(By.XPATH, '/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]/span')
billionaires.click()

# Open world billionaires page
world_billionaires = driver.find_element(By.XPATH, '/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[2]/div[3]/ul/li[1]/a')
world_billionaires.click()

# Wait for the page to load
time.sleep(10)

In [48]:
# Get details of all billionaires on the page
billionaires_rank = []
billionaires_name = []
billionaires_net_worth = []
billionaires_age = []
billionaires_citizenship = []
billionaires_source = []
billionaires_industry = []

for i in range(1, 11):
    rank_xpath = f"/html/body/div[1]/div[1]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div[2]/div[1]/div[1]/div"
    name_xpath = f"/html/body/div[1]/div[1]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div[2]/div[2]/div[1]/div"
    net_worth_xpath = f"/html/body/div[1]/div[1]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[1]/div"
    age_xpath = f"/html/body/div[1]/div[1]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div[2]/div[4]/div[1]/div"
    citizenship_xpath = f"/html/body/div[1]/div[1]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div[2]/div[5]/div[1]/div"
    source_xpath = f"/html/body/div[1]/div[1]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div[2]/div[6]/div[1]/div"
    industry_xpath = f"/html/body/div[1]/div[1]/div/div[2]/div[3]/div[2]/div[2]/div[1]/div/div[2]/div[7]/div[1]/div"
    
    billionaires_rank.append(driver.find_element(By.XPATH, rank_xpath).text)
    billionaires_name.append(driver.find_element(By.XPATH, name_xpath).text)
    billionaires_net_worth.append(driver.find_element(By.XPATH, net_worth_xpath).text)
    billionaires_age.append(driver.find_element(By.XPATH, age_xpath).text)
    billionaires_citizenship.append(driver.find_element(By.XPATH, citizenship_xpath).text)
    billionaires_source.append(driver.find_element(By.XPATH, source_xpath).text)
    billionaires_industry.append(driver.find_element(By.XPATH, industry_xpath).text)

In [49]:
# Create a Pandas dataframe and save it to a CSV file
df = pd.DataFrame({
    "Billionaires Rank": billionaires_rank,
    "Billionaires Name": billionaires_name,
    "Billionaires Net Worth": billionaires_net_worth,
    "Billionaires Age": billionaires_age,
    "Billionaires Citizenship": billionaires_citizenship,
    "Billionaires Source" : billionaires_source,
    "Billionaires Industry": billionaires_industry})
print(df)

df.to_csv("billionaires.csv", index=False)

   Rank  Name  Net Worth  Age          Citizenship  Source  Industry
0  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
1  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
2  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
3  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
4  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
5  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
6  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
7  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
8  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY
9  RANK  NAME  NET WORTH  AGE  COUNTRY / TERRITORY  SOURCE  INDUSTRY


In [50]:
driver.quit()

YOUTUBE-VIDEO 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Open the YouTube video
driver = webdriver.Chrome()
driver.get("https://www.youtube.com/watch?v=e-ORhEE9VVg")

# Scroll down to generate more comments
for _ in range(1000):
    driver.execute_script("window.scrollBy(0, 1000)")

# Extract comments, upvotes, and time posted
comments = driver.find_elements(By.XPATH, "//yt-formatted-string[@class='style-scope ytd-comment-renderer']")
upvotes = driver.find_elements(By.XPATH, "//span[@class='style-scope ytd-comment-action-buttons-renderer']//span[1]")
time_posted = driver.find_elements(By.XPATH, "//a[@class='yt-simple-endpoint style-scope yt-formatted-string']//span")

# Create a DataFrame from the extracted data
comments_list = []
upvotes_list = []
time_posted_list = []

for comment, upvote, time in zip(comments, upvotes, time_posted):
    comments_list.append(comment.text)
    upvotes_list.append(upvote.text)
    time_posted_list.append(time.text)

data = {
    'Comment': comments_list,
    'Upvote': upvotes_list,
    'Time Posted': time_posted_list}

df = pd.DataFrame(data)
print(df)

# Save the data to a CSV file
df.to_csv('youtube_comments.csv', index=False)


Empty DataFrame
Columns: [Comment, Upvote, Time Posted]
Index: []


In [2]:
# close the drive
driver.quit()

HOSTELSWORLD

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(r"/Users/sukanyasemwal/Desktop/chromedriver.exe")
driver.get("https://www.hostelworld.com/")
time.sleep(3)

#Click on bar
bar=driver.find_element(By.XPATH,'//input[@id="location-text-input-field"]')
bar.click()
time.sleep(2)

#search london
search=driver.find_element(By.XPATH,'//input[@id="search-input-field"]')
search.send_keys("London")
time.sleep(5)

#Click on london name
london=driver.find_element(By.XPATH,'//ul[@id="predicted-search-results"]/li[2]/div')
london.click()
time.sleep(2)

# Click on "Let's Go" button
lets_go_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(@id, 'search-button')]")))
lets_go_button.click()

/var/folders/xm/ntpbjvjx7cbg4ts7grjkd9r40000gn/T/ipykernel_1815/410093053.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"/Users/sukanyasemwal/Desktop/chromedriver.exe")


In [4]:
# Extract search results
hostel_names = []
hostel_ratings = []
hostel_prices = []


results = driver.find_elements(By.XPATH, '//div[@class="property-card"]')
for result in results:
    name = result.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[4]/div/div/div[3]/div[3]/div[1]/h2/a').text
    rating = result.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[4]/div/div/div[3]/div[3]/div[2]/div[2]/a').text
    price = result.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[4]/div/div/div[3]/div[4]')
    hostel_names.append(name)
    hostel_ratings.append(rating)
    hostel_prices.append(price)

In [5]:
# Create a DataFrame with the search results
data = {"Hostel Name": hostel_names, "Rating": hostel_ratings, "Price": hostel_prices}
df = pd.DataFrame(data)
print(df)

# Save the DataFrame to a CSV file
df.to_csv("hostel_results.csv", index=False)

# Close the Chrome driver
driver.quit()

Empty DataFrame
Columns: [Hostel Name, Rating, Price]
Index: []
